In [1]:
import os
from matplotlib import pyplot as plt
import pickle
import networkx as nx

import json



def read_data(fname):
    # Initiate
    nodes = dict()
    times = dict()

    # Create Graph
    #G = nx.Graph()
    # Read stream
    all_t = []
    f = open(os.path.join(fname), "r")
    lines = f.readlines()
    f.close()
    nb_lines = 0
    #start_time = TM.time()
    for line in lines:
        t, u, v = line.strip().split(' ')
        t = int(t)
        
        link = frozenset([u, v])

        ##Add the edge in the static graph G
        #G.add_edge(str(u), str(v))

        # Populate data structures
        if link not in times:
            times[link] = []
        times[link].append(t)
        
        if u not in nodes:
            nodes[u] = set()

        if v not in nodes:
            nodes[v] = set()

        nodes[u].add(v)
        nodes[v].add(u)

        nb_lines += 1
        
        all_t.append(t)
        
    print("Number of links: {}".format(nb_lines))
    print("Number of Nodes: {}".format(len(nodes)))
    print("Number of Edges: {}".format(len(times)))
    print("start time: {}".format(min(all_t)) )
    print("end time: {}".format(max(all_t)) )
    
    return nodes, times


In [2]:
import copy
import bisect

In [3]:
def computeNeighborhood( G, begin, lifetime, delta):
    neighborhood = dict()
    neighbors = dict()
    for vw in G.keys():
        if len(vw) > 1:
            intervals = []
            edgeList = sorted(G[vw])

            start = max(begin, edgeList.pop(0) - delta)
            end = min(begin + lifetime, start + delta * 2)
            for t in edgeList:
                if (end+1) < t:
                    intervals.append((start,end))
                    start = max(begin, t - delta)
                    end = min(begin + lifetime,t + delta)
                else:
                    end = min(begin + lifetime, t + delta)
            intervals.append((start,end))
            neighborhood[vw] = intervals
            
            #print(intervals)
            vwList = list(vw)
            #print(vwList)
            for i in range(len(vwList)):
                if vwList[i] not in neighbors:
                    neighbors[vwList[i]] = [vwList[(i + 1) % 2]]
                    #print("if**", i, (i + 1) % 2, vwList[i], vwList[(i + 1) % 2], neighbors[vwList[i]]) 
                else:
                    neighbors[vwList[i]].append(vwList[(i + 1) % 2])
                    #print("else**", i, (i + 1) % 2, vwList[i], vwList[(i + 1) % 2], neighbors[vwList[i]]) 
    return neighbors, neighborhood

In [4]:
def computeNeighborhood_gamma( G, begin, lifetime, delta, gamma):
    neighborhood = dict()
    neighbors = dict()
    for vw in G.keys():
        if len(vw) > 1:
            intervals = []
            edgeList = sorted(G[vw])
            ### Changed code - start: Bithika
            if len(edgeList) >= gamma:
                for i in range(len(edgeList)):
                    if i == 0:
                        current_list = []
                        current_list.append(edgeList[i])
                        #print(current_list, i)
                        #print('if : case 1')
                    elif len(current_list) < gamma:
                        if (edgeList[i] - current_list[0]) <= delta:  
                            current_list.append(edgeList[i])
                            #print(current_list, i)
                            #print('elif : case 2')  
                        else:
                            current_list = []
                            links_1 = G[vw][bisect.bisect_left(G[vw], edgeList[i] - delta):bisect.bisect_right(G[vw], edgeList[i])]
                            for l in links_1:
                                current_list.append(l)

                    elif len(current_list) >= gamma :
                        if (current_list[len(current_list)-gamma]+1+delta) >= edgeList[i]:
                            current_list.append(edgeList[i])
                        else:
                            tb = current_list[gamma-1] - delta
                            te = current_list[len(current_list) - gamma] + delta
                            intervals.append((tb,te))
                            current_list = []
                            links_1 = G[vw][bisect.bisect_left(G[vw], edgeList[i] - delta):bisect.bisect_right(G[vw], edgeList[i])]
                            for l in links_1:
                                current_list.append(l)
                                #print(current_list, i)
                                #print('elif else : case 4')
                    #else:
                        #None

                    if (i== len(edgeList)-1) and len(current_list) >= gamma:
                        #print(current_list, i)
                        tb = current_list[0+gamma-1] - delta
                        te = current_list[len(current_list) - gamma] + delta
                        intervals.append((tb,te))
                        current_list = []
                        #print('if : case 6')
                        
            ### Changed code - end: Bithika            
            neighborhood[vw] = intervals
            #print(intervals)
            vwList = list(vw)
            #print(vwList)
            for i in range(len(vwList)):
                if vwList[i] not in neighbors:
                    neighbors[vwList[i]] = [vwList[(i + 1) % 2]]
                else:
                    neighbors[vwList[i]].append(vwList[(i + 1) % 2])
    return neighbors, neighborhood

In [4]:
os.getcwd()


'C:\\Users\\hp\\Code\\Delta-Gamma-Clique\\Snapshot-based-enumeration\\Methods\\BKDeltaGamma'

In [5]:
rootdir = "C:\\Users\\hp\\Code\\Delta-Gamma-Clique\\weak_clique\\"
dataset = "PrimarySchool"
filename = "primaryschool_processed.txt"
data_file = os.path.join(rootdir, "Dataset", dataset, filename)

In [6]:
nodes, times = read_data(data_file)

Number of links: 125773
Number of Nodes: 242
Number of Edges: 8317
start time: 11561
end time: 17406


In [7]:
delta = 30

In [47]:
neighbors, neighborhood = computeNeighborhood(times, begin=11561-30, lifetime=17406+30-(11561-30), delta=30)

In [48]:
#times
neighbors_gamma, neighborhood_gamma = computeNeighborhood_gamma(times, begin=11561-30, lifetime=17406+30-(11561-30), delta=30, gamma=1)

In [49]:
c=0
c1=0
for i in neighborhood.keys():
    for j in neighborhood[i]:
        c1 +=1
        if j not in neighborhood_gamma[i]:
            print(i, j)
            c+= 1
print(c, c1)

0 31002


In [38]:
(17406-17362)

44

In [41]:
times[frozenset({'51', '211'})]

[17392]

In [40]:
neighborhood[frozenset({'51', '211'})]

[(17362, 17406)]

In [39]:
neighborhood_gamma[frozenset({'51', '211'})]

[(17362, 17422)]

In [20]:
11643- 11710

-67

In [23]:
11667 - 11643

24

In [22]:
11667-30

11637

In [11]:
class Clique:

    def __init__(self, V, I):
        (start, end) = I;
        self.R = frozenset(V);
        self.startInterval = start;
        self.endInterval = end;

    def __eq__(self, other):
        return self.R == other.R and self.startInterval == other.startInterval and self.endInterval == other.endInterval

    def __hash__(self):
        return hash((self.R,self.startInterval, self.endInterval))

    def __str__(self):
        return "{"+','.join(map(str, list(self.R))) + "} (" + str(self.startInterval) + "," + str(self.endInterval)+")"

    def expandClique(self, v, I):
        (start, end) = I
        self.R |= frozenset([v])
        if start <= end:
            self.startInterval = start
            self.endInterval = end

    def expandCliqueByVertex(self, v):
        self.R |= frozenset([v])

    def size(self):
        return len(self.R)

In [21]:
from Clique import Clique
from Clique_2p import Clique

In [12]:
import os
os.getcwd()

'/home/rs/19CS92R05/Work_With_Suman/Delta-Gamma-Clique/Snapshot-based-enumeration/Methods/BKDeltaGamma/trial'

In [15]:
import pickle
with open("cliqueset_InfectiousSmall_BKdeltagamma_30_10.pkl", "rb") as file:
    CS = pickle.load(file)

In [16]:
B=set()
for c in CS:
    B.add((c.R, (c.startInterval, c.endInterval)))

In [17]:
B

{(frozenset({'33', '35'}), (2814, 2862)),
 (frozenset({'374', '375'}), (3736, 3791)),
 (frozenset({'17', '18'}), (2615, 2713)),
 (frozenset({'345', '346'}), (3886, 3920)),
 (frozenset({'367', '396'}), (3949, 3990)),
 (frozenset({'50', '51'}), (3029, 3125)),
 (frozenset({'38', '39'}), (2808, 2844)),
 (frozenset({'56', '58'}), (2846, 2887)),
 (frozenset({'112', '113'}), (2902, 2958)),
 (frozenset({'74', '78'}), (2861, 2922)),
 (frozenset({'400', '401'}), (3964, 4009)),
 (frozenset({'33', '36'}), (2738, 2774)),
 (frozenset({'33', '35', '36'}), (2815, 2862)),
 (frozenset({'41', '42', '44'}), (2741, 2776)),
 (frozenset({'272', '369'}), (3639, 3695)),
 (frozenset({'339', '340'}), (3659, 3706)),
 (frozenset({'71', '73'}), (2871, 2937)),
 (frozenset({'215', '217'}), (3141, 3176)),
 (frozenset({'354', '355'}), (3564, 3688)),
 (frozenset({'42', '44'}), (2739, 2778)),
 (frozenset({'87', '88'}), (2857, 2894)),
 (frozenset({'112', '113'}), (3027, 3095)),
 (frozenset({'200', '204'}), (3149, 3191)),


In [24]:
with open("/home/rs/19CS92R05/Work_With_Suman/Delta-Gamma-Clique/Snapshot-based-enumeration/Result/Infectious/TwoPhase/maximal_clique_set_30_10.pkl", "rb") as file:
    AS = pickle.load(file)

In [25]:
A=set()
for c in AS:
    A.add((c._X, (c._tb, c._te)))

In [26]:
A-B

set()

In [27]:
B-A

set()

In [28]:
from collections import defaultdict
A_dic = dict()
for k in range(2,10):
    A_dic[k] = defaultdict(list)
for c in A:   
    X = list(c[0])
    X = [int(i) for i in X]
    A_dic[len(X)][frozenset(X)].append(tuple([c[1][0], c[1][1]]))    

In [29]:
A_dic

{2: defaultdict(list,
             {frozenset({33, 35}): [(2814, 2862)],
              frozenset({374, 375}): [(3736, 3791),
               (3775, 3968),
               (3763, 3803)],
              frozenset({17, 18}): [(2615, 2713)],
              frozenset({345, 346}): [(3886, 3920),
               (3717, 3781),
               (3650, 3709),
               (3682, 3720)],
              frozenset({367, 396}): [(3949, 3990),
               (3905, 3936),
               (3830, 3860),
               (3932, 3975),
               (3857, 3931),
               (3817, 3857)],
              frozenset({50, 51}): [(3029, 3125),
               (3008, 3040),
               (3015, 3046),
               (2889, 2995),
               (2732, 2899),
               (3022, 3055)],
              frozenset({38, 39}): [(2808, 2844), (2710, 2763), (2802, 2834)],
              frozenset({56, 58}): [(2846, 2887)],
              frozenset({112, 113}): [(2902, 2958),
               (3027, 3095),
               (3105

In [30]:

B_dic = dict()
for k in range(2,10):
    B_dic[k] = defaultdict(list)
for c in B:   
    X = list(c[0])
    X = [int(i) for i in X]
    B_dic[len(X)][frozenset(X)].append(tuple([c[1][0], c[1][1]]))    

In [31]:
B_dic

{2: defaultdict(list,
             {frozenset({33, 35}): [(2814, 2862)],
              frozenset({374, 375}): [(3736, 3791),
               (3775, 3968),
               (3763, 3803)],
              frozenset({17, 18}): [(2615, 2713)],
              frozenset({345, 346}): [(3886, 3920),
               (3717, 3781),
               (3650, 3709),
               (3682, 3720)],
              frozenset({367, 396}): [(3949, 3990),
               (3905, 3936),
               (3830, 3860),
               (3932, 3975),
               (3857, 3931),
               (3817, 3857)],
              frozenset({50, 51}): [(3029, 3125),
               (3008, 3040),
               (3015, 3046),
               (2889, 2995),
               (2732, 2899),
               (3022, 3055)],
              frozenset({38, 39}): [(2808, 2844), (2710, 2763), (2802, 2834)],
              frozenset({56, 58}): [(2846, 2887)],
              frozenset({112, 113}): [(2902, 2958),
               (3027, 3095),
               (3105

In [32]:
correct_1 = True
for k in A_dic.keys():
    for X in A_dic[k].keys():
        for I in A_dic[k][X]:
            if X not in B_dic[k]:
                correct_1 = False
                print(X, "not in B")
            else:
                if I not in B_dic[k][X]:
                    correct_1 = False

In [33]:
correct_2 = True
for k in B_dic.keys():
    for X in B_dic[k].keys():
        for I in B_dic[k][X]:
            if X not in A_dic[k]:
                correct_2 = False
            else:                
                if I not in A_dic[k][X]:
                    correct_2 = False

In [34]:
if correct_1 == True and correct_2 == True:
    print("Correctness proved")
elif correct_1 == True and correct_2 == False:
    print("Correctness Failed: Proposed method results - non-maximal clique")
elif correct_1 == False and correct_2 == True:
    print("Correctness Failed: Proposed method misses - maximal clique")
else:
    print("Correctness Failed")

Correctness proved
